In [9]:
import pandas as pd
import numpy as np
import csv
import time
from datetime import datetime, timedelta
# from ics import Calendar, Event

soft_acts = []
hard_acts = []

# Assuming your file is in the same directory as your Python script or Jupyter notebook
file_path = "data-2023.csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Set 'duration' as integers
df['duration'] = df['duration'].astype(int)

# Display rows where 'is_pro' is not NaN
df_soft_skills = df[~df['is_pro'].isna()]

# Reset the index after filtering
df_soft_skills.reset_index(drop=True, inplace=True)

# Drop rows where 'is_ignored' is True
df = df[df['is_ignored'] != True]

# Reset the index after dropping rows
df.reset_index(drop=True, inplace=True)

# Drop rows where 'is_pro' is na
df = df[df['is_pro'].isna()]

# Reset the index after dropping rows
df.reset_index(drop=True, inplace=True)

# Remove specified columns
columns_to_remove = ['day_week_n', 'week', 'is_ignored', 'is_pro']
df = df.drop(columns=columns_to_remove, axis=1)

# Drop the last row (exam day)
df_hard_skills = df.drop(df.index[-1])

# Reset the index after dropping the last row
df_hard_skills.reset_index(drop=True, inplace=True)

soft_acts = df_soft_skills
hard_acts = df_hard_skills

# Display the modified DataFrame
display(hard_acts)


,title,duration,module
0,Opening,20,Introduction
1,Welcome & Expectations,45,Introduction
2,Program Overview and Introductions,60,Introduction
3,Curriculum Overview,30,Introduction
4,Vendor Introduction,45,Introduction
...,...,...,...
306,Mock exam and review,90,Ending and exam preparation
307,Review product FAQ / review AWS /\nAWS whitepa...,80,Ending and exam preparation
308,Review product FAQ / review AWS /\nAWS whitepa...,85,Ending and exam preparation
309,Mock exam and review,80,Ending and exam preparation


In [10]:
MAX_MINUTES_DAY = 200
TICKET_DURATION = 5
RECAP_DURATION = 5
TOTAL_FLEX_TIME = 0

class Day:
    def __init__(self, date_start, module_name):
        self.date_start = date_start
        self.date_end = date_start + timedelta(minutes=MAX_MINUTES_DAY)
        self.module_name = module_name
        self.activities = [{'title':'Recap', 'duration':RECAP_DURATION}]
        self.total_duration = RECAP_DURATION

    def can_add_hard(self, activity):
        return (
            self.total_duration + activity['duration'] <= MAX_MINUTES_DAY - TICKET_DURATION
            and self.module_name == activity['module']
        )

    def can_add_soft(self, activity):
        return (
            self.total_duration + activity['duration'] <= MAX_MINUTES_DAY - TICKET_DURATION
        )

    def add_act(self, activity):
        self.activities.append({'title':activity['title'], 'duration':activity['duration']})
        self.total_duration += activity['duration']

    def add_ticket(self):
        self.activities.append({'title':'Exit Ticket', 'duration':TICKET_DURATION})
        self.total_duration += TICKET_DURATION

    def add_flex_time(self):
        flex_time = MAX_MINUTES_DAY - self.total_duration
        if flex_time > 0:
            self.activities.append({'title':'Flex Time', 'duration':flex_time})
            self.total_duration += flex_time
        return(flex_time)
        
    def __str__(self):
        return (
            f"{self.module_name} - {self.date_start}"
        )

# Function to parse date from the CSV file
def parse_date(date_str):
    return datetime.strptime(date_str, '%d/%m/%Y %H:%M')

# Function to check if a given date is a skipped day (Friday, Sunday or Holidays)
def is_skipday(date):
    # Check if it's a Friday or Sunday
    if date.weekday() in [4, 6]:
        return True
    # Check if it's a holiday
    holidays = [
        '01/01/2024', '06/01/2024', '25/03/2024', '28/03/2024', '29/03/2024', 
        '01/04/2024', '01/05/2024',
        '20/05/2024', '24/06/2024', '15/08/2024', '11/09/2024', '24/09/2024',
        '12/11/2024', '01/11/2024', '06/12/2024', '25/12/2024', '26/12/2024',
        '30/03/2024', '18/05/2024', '22/06/2024'
    ]
    return date.strftime('%d/%m/%Y') in holidays

def is_saturday(date):
    return date.weekday() == 5  # Saturday

# Function to add a day to a given date, skipping skipdays
def add_weekday(date, days):
    new_date = date + timedelta(days)
    while is_skipday(new_date):
        new_date += timedelta(1)
    if is_saturday(new_date):
        new_date = new_date.replace(hour=10, minute=30)
    else:
        new_date = new_date.replace(hour=18, minute=0)
    return new_date

def get_first(activities):
    if not activities.empty:
        return (activities.iloc[0])

def remove_first(activities):
    return activities.iloc[1:]

days = []

start_date = parse_date('16/01/2024 18:00')

# Iterate through the DataFrame until it's empty
while not (hard_acts.empty and soft_acts.empty):
    
    hard_act = get_first(hard_acts)
    soft_act = get_first(soft_acts)

    if (not hard_acts.empty):
        module_name = hard_act['module']
    else:
        module_name = "Soft skills"
    day = Day(start_date, module_name)
    
    while (not hard_acts.empty and day.can_add_hard(hard_act)):
        day.add_act(hard_act)
        hard_acts = remove_first(hard_acts)
        hard_act = get_first(hard_acts)
    
    while (not soft_acts.empty and day.can_add_soft(soft_act)):
        day.add_act(soft_act)
        soft_acts = remove_first(soft_acts)
        soft_act = get_first(soft_acts)
    
    day.add_ticket()
    TOTAL_FLEX_TIME += day.add_flex_time()
    days.append(day)
    
    start_date = add_weekday(start_date, 1)
    if (start_date.date() == datetime(2024, 3, 14).date()):
        MAX_MINUTES_DAY = 220
    print(str(day))



# The DataFrame is now empty
print("DataFrame is empty., Total flex is ", TOTAL_FLEX_TIME)

# (total minutes, recap duration, ticket duration) total flex time in minutes. last day
# (180 10 10) Case without separating soft and hard skills: 5855 total flex time. 2024-08-24
# (180 10 10) Case with separating soft and hard skills: 5695 total flex time. 2024-08-24
# (200 5 5) Case with separating soft and hard skills: 4715 total flex time. 2024-07-13
# (200 5 5) Case with separating soft and hard skills: 4530 total flex time. 2024-07-11

Introduction - 2024-01-16 18:00:00
Introduction - 2024-01-17 18:00:00
Cloud Foundations - 2024-01-18 18:00:00
Cloud Foundations - 2024-01-20 10:30:00
Cloud Foundations - 2024-01-22 18:00:00
Cloud Foundations - 2024-01-23 18:00:00
Cloud Foundations - 2024-01-24 18:00:00
Cloud Foundations - 2024-01-25 18:00:00
Cloud Foundations - 2024-01-27 10:30:00
Linux - 2024-01-29 18:00:00
Linux - 2024-01-30 18:00:00
Linux - 2024-01-31 18:00:00
Linux - 2024-02-01 18:00:00
Linux - 2024-02-03 10:30:00
Linux - 2024-02-05 18:00:00
Linux - 2024-02-06 18:00:00
Linux - 2024-02-07 18:00:00
Linux - 2024-02-08 18:00:00
Linux - 2024-02-10 10:30:00
Linux - 2024-02-12 18:00:00
Networking - 2024-02-13 18:00:00
Networking - 2024-02-14 18:00:00
Networking - 2024-02-15 18:00:00
Networking - 2024-02-17 10:30:00
Networking - 2024-02-19 18:00:00
Networking - 2024-02-20 18:00:00
Networking - 2024-02-21 18:00:00
Networking - 2024-02-22 18:00:00
Security - 2024-02-24 10:30:00
Security - 2024-02-26 18:00:00
Security - 2024-

In [11]:
import csv
from datetime import datetime, timedelta

# CSV file path
csv_file_path = 'output.csv'

# Writing to CSV
with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['module_name', 'full_date','day','time_start', 'time_end', 'day_plan']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write header
    writer.writeheader()

    # Write data
    for a_day in days:
        writer.writerow({
            'full_date': a_day.date_start.strftime('%d/%m/%Y'),
            'time_start': a_day.date_start.strftime('%H:%M'),
            'time_end': a_day.date_end.strftime('%H:%M'),
            'day': a_day.date_start.strftime('%A'),
            'module_name': a_day.module_name,
            'day_plan': ' | '.join(f"{d['title']} - {d['duration']} min" for d in a_day.activities)
        })

print(f'Data has been written to {csv_file_path}.')


Data has been written to output.csv.
